In [4]:
from sklearn.feature_extraction.text import CountVectorizer
import string
import nltk
from nltk.corpus import stopwords
stopwords_en = set(stopwords.words('english'))
import pickle
import re

In [5]:
dataset_folder = 'dbpedia_100k_test'
texts = []
entities = []
with open('../contextualized_topic_models/data/' + dataset_folder + '/new_training_sets', 'rb') as fr:
    data = pickle.load(fr)

In [6]:
for k,v in data.items():
    entities.append(k)
    texts.append(v)
        

In [19]:
def preprocessing(corpus, entities, stop_words):
    corpus = [doc.replace("\\n*", " ") for doc in corpus]
    corpus = [doc.replace("\\", " ") for doc in corpus]

    corpus = [' '.join(doc.split(" ")[:400]) for doc in corpus]
    unprepr_corpus = corpus
    
    corpus = [doc.lower() for doc in corpus]
    corpus = [doc.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))) for doc in corpus]
    #corpus = [doc.translate(str.maketrans(string.digits, ' '*len(string.digits))) for doc in corpus]
    corpus = [' '.join([w for w in doc.split() if len(w)>0 and w not in stop_words]) for doc in corpus]
    vectorizer = CountVectorizer(max_features=2000, token_pattern=r'\b[a-zA-Z]{2,}\b')
    X = vectorizer.fit_transform(corpus)
    vocab = set(vectorizer.get_feature_names())
    corpus = [' '.join([w for w in doc.split() if w in vocab]) for doc in corpus]
    
    new_corpus, new_unprep_corpus, new_entities = [], [], []
    for i, doc in enumerate(corpus):
        if len(doc)>0:
            new_corpus.append(doc)
            new_unprep_corpus.append(unprepr_corpus[i])
            new_entities.append(entities[i])
    
    return new_corpus, new_unprep_corpus, new_entities, vocab

In [20]:
preprocessed_corpus, unpreprocessed_corpus, new_entities, vocab = preprocessing(texts, entities, stopwords_en)

In [21]:
texts[10:30]
preprocessed_corpus[10:30]

['persian also also known village rural district central district county province iran census population families',
 'college also known faculty university new new canada college program bachelor philosophy leadership leadership program canada college also philosophy bachelor studies programs',
 'indian tamil language film directed written produced entertainment film stars lead roles roles film score composed began production july',
 'associated number concept commonly applied also used possible may first number states usually unique certain assigned energy side chain commonly used base may used used statistical possible',
 'secretary drama film directed produced productions film based novel name film stars lead roles music composed',
 'science fiction novel young adult literature author takes place united states civil war near future civil war known second civil war war fought reached parents order children',
 'family found western habitat consists high mountains adults recorded wing 

In [22]:
len(unpreprocessed_corpus)

149699

In [23]:
filtered_unpreprocessed_corpus, filtered_preprocessed_corpus = [], []
for i, doc in enumerate(unpreprocessed_corpus):
    #if len(doc)>50:
    filtered_unpreprocessed_corpus.append(doc)
    filtered_preprocessed_corpus.append(preprocessed_corpus[i])

In [24]:
test_num = 0
output_path = '../contextualized_topic_models/data/' + dataset_folder + "//"

with open(output_path +"dbpedia_train_unprep.txt", "w", encoding="utf8") as fw:
    for l in filtered_unpreprocessed_corpus[test_num:]:
        fw.write(l.strip() + "\n")
        
with open(output_path +"dbpedia_train_prep.txt", "w", encoding="utf8") as fw:
    for l in filtered_preprocessed_corpus[test_num:]:
        fw.write(l.strip() + "\n")
        
with open(output_path +"dbpedia_test_unprep.txt", "w", encoding="utf8") as fw:
    for l in filtered_unpreprocessed_corpus[:test_num]:
        fw.write(l.strip() + "\n")
        
with open(output_path +"dbpedia_test_prep.txt", "w", encoding="utf8") as fw:
    for l in filtered_preprocessed_corpus[:test_num]:
        fw.write(l.strip() + "\n")
        
with open(output_path +"dbpedia_test_entities.txt", "w", encoding="utf8") as fw:
    for l in new_entities[:test_num]:
        fw.write(l.strip() + "\n")
with open(output_path +"dbpedia_train_entities.txt", "w", encoding="utf8") as fw:
    for l in new_entities[test_num:]:
        fw.write(l.strip() + "\n")

In [35]:
dataset = 'dbpedia_100k_test'
with open('../contextualized_topic_models/data/' + dataset + '/new_test_sets', 'rb') as fr:
    data = pickle.load(fr)
entities = [k for k in data.keys()]
test_docs = [data[k] for k in data.keys()]

 

In [38]:
f = 200

test_docs = [[" ".join(d.split(" ")[:f]) for d in doc] for doc in test_docs]
with open('../contextualized_topic_models/data/' + dataset + '/test_set_' + str(f), 'wb') as fw:
    pickle.dump(test_docs , fw)